Just a script to embed all of the textual and image data in CLIP and save the embeddings in the drive folder
Requires the image data to be unzipped, and all .ini files do be deleted.

### Setup

In [4]:
%pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to c:\users\user\appdata\local\temp\pip-req-build-l71ucxe0
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\user\AppData\Local\Temp\pip-req-build-l71ucxe0'


In [5]:
# ----- stdlib & deps ------------------------------------------------------
import os, sys, numpy as np, torch
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import clip
# -------------------------------------------------------------------------

# ----- paths --------------------------------------------------------------
drive_root = Path(r"G:/")
PROJ_DIR   = drive_root / ".shortcut-targets-by-id/1CwmFOsYFnq6t33KAzpvw0gaOTQXbcozs/brain-decoder-files"
images_dir = PROJ_DIR / "experiment-images"

text_npz   = PROJ_DIR / "clip_text_embeddings.npz"
image_npz  = PROJ_DIR / "clip_image_embeddings.npz"   # new: raw vectors
assert images_dir.is_dir(), f"{images_dir} not found!"

# ----- load CLIP model ----------------------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14@336px", device=device)  # returns 768-d vectors
model.eval()

# ----- concept labels -----------------------------------------------------
concepts = np.genfromtxt(PROJ_DIR / "concepts.txt", dtype=str)


### Embed textual data

In [ ]:
with torch.no_grad():
    toks = clip.tokenize([f"A photo of {c}" for c in concepts]).to(device)
    txt  = model.encode_text(toks)
    txt  = txt / txt.norm(dim=-1, keepdim=True)

np.savez_compressed(text_npz, data=txt.cpu().numpy().astype(np.float32))
print("Text vectors saved to", text_npz)

### Embed image data

In [6]:
img_paths = [(con, p) for con in concepts for p in (images_dir / con).glob("*.jpg")]

BATCH = 64
all_vecs, all_cons, all_files = [], [], []

for i in tqdm(range(0, len(img_paths), BATCH), desc="encoding images"):
    batch_paths = img_paths[i : i + BATCH]

    imgs = [preprocess(Image.open(p).convert("RGB")) for _, p in batch_paths]
    imgs = torch.stack(imgs).to(device)

    with torch.no_grad():
        vecs = model.encode_image(imgs).float()
    vecs = vecs / vecs.norm(dim=-1, keepdim=True)

    all_vecs.append(vecs.cpu())
    all_cons.extend([con for con, _ in batch_paths])
    all_files.extend([p.name for _, p in batch_paths])

embeddings = torch.cat(all_vecs).numpy().astype(np.float32)

np.savez_compressed(
    image_npz,
    embeddings=embeddings,
    concepts=np.array(all_cons),
    filenames=np.array(all_files),
)
print(f"Image vectors saved to ", image_npz)


encoding images: 100%|██████████| 17/17 [36:01<00:00, 127.12s/it]

Image vectors saved to  G:\.shortcut-targets-by-id\1CwmFOsYFnq6t33KAzpvw0gaOTQXbcozs\brain-decoder-files\clip_image_embeddings.npz
